In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from openai import OpenAI
from together import Together

tqdm.pandas()

In [ ]:
os.environ['OPENAI_API_KEY'] = "sk-proj-SWPkbjlG7ZO9tAKANL69T3BlbkFJ3iiUUZHszD9AuQnEjhLE"
os.environ['TOGETHER_API_KEY'] = "90e216b11fbd62f9b0ce3a9666cbcb134a116857b3b62f95b6643bde91feb364"

try:
    openai_client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'] 
    )

    togetherai_client = Together(
        api_key=os.environ.get("TOGETHER_API_KEY")
    )
except Exception as e:
    print(f"Please specify your OPENAI and TOGETHER-AI keys for inference")

def predict(
      prompt, 
      system_description,
      use_temperature_samping=False,
      model_name="mistralai/Mistral-7B-Instruct-v0.3"
    #   model_name="mistralai/Mistral-7B-Instruct-v0.3"

):
    temperature = 0.5 if not use_temperature_samping else random.uniform(0, 0.5)

    if 'gpt' in model_name:
        response = openai_client.chat.completions.create(
        model=model_name,
        temperature=temperature,
        top_p=1,
        response_format = { "type": "json_object"},
        messages = [
                {"role": "system", "content": system_description},
                {"role": "user", "content": prompt},
            ],
        )
        content = response.choices[0].message.content
        return content
    
    else:
        response = togetherai_client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_description},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=1024,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["</s>"],
            logprobs=False,
            n=1
        )
        content = response.choices[0].message.content
        return content
        

def predict_openai(
      prompt, 
      system_description,
      use_temperature_samping=False,
      model_name="gpt-4o"

):
    temperature = 0.5 if not use_temperature_samping else random.uniform(0, 0.5)

    response = openai_client.chat.completions.create(
    model=model_name,
    temperature=temperature,
    top_p=1,
    response_format = { "type": "json_object"},
    messages = [
            {"role": "system", "content": system_description},
            {"role": "user", "content": prompt},
        ],
    )
    content = response.choices[0].message.content
    return content

def predict_togetherai(
        input_prompt, 
        system_description, 
        use_temperature_samping=False,
        model_name="mistralai/Mistral-7B-Instruct-v0.3"
    ):

    temperature = 0.5 if not use_temperature_samping else random.uniform(0, 0.5)

    response = togetherai_client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_description},
            {"role": "user", "content": input_prompt}
        ],
        temperature=temperature,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        logprobs=False,
        n=1
    )

    return response.choices[0].message.content

In [ ]:
import json
import os
import random
import re
import string

import numpy as np
import pandas as pd
import torch
from cleantext import clean
from loguru import logger
from sklearn.model_selection import train_test_split

dashed_line = "- " * 50

def clean_text(text: str, min_len=4) -> str:
    """
    Returns cleaned text (post/title) by removing extra whitespaces, unprintable characters, recurring punctions, & urls

    Before:

    My ðŸ¤ [ndad] (https://np.reddit.com/r/raisedbynarcissists/comments/4puzqb/if_ndad_shows_up_to_my_graduation_im_going_to/) and I have been estranged for about 4-5 months now. Since the estrangement, I have noticed feeling this desire to tighten my abdominal muscles when I remember something my ndad did or didn't do that led me to feel so much pain as a result of feeling angry, hurt or depressed by all the hurt he has caused me.

    The ¤ only other time I think of tightening my abdominal muscles is during exercise because I remember when I used exercise videos, the instructor would instruct such...!

    I'm taking medications ([at the very least, I am trying](https://np.reddit.com/r/Anger/comments/4rjatz/new_doctor_wants_me_to_go_off_antidepressants/)) and am going to therapy so I hope I am handling this pretty well, just wanted to get second opinions.

    Should I be concerned about this.......? Do you have any idea why one would desire to tighten abdominal muscles in this situation or in situations such as these......!?
    Ÿ


    *********************************************************************************************************************************

    After:
    my [ndad] (<url>) and i have been estranged for about 4-5 months now. since the estrangement, i have noticed feeling this desire to tighten my abdominal muscles when i remember something my ndad did or didn't do that led me to feel so much pain as a result of feeling angry, hurt or depressed by all the hurt he has caused me. the  only other time i think of tightening my abdominal muscles is during exercise because i remember when i used exercise videos, the instructor would instruct such! i'm taking medications ([at the very least, i am trying](<url>)) and am going to therapy so i hope i am handling this pretty well, just wanted to get second opinions. should i be concerned about this? do you have any idea why one would desire to tighten abdominal muscles in this situation or in situations such as these?

    """

    punc = """()-[]{};:'"\<>/@#$%^&*_~"""

    if not isinstance(text, str):
        logger.debug(ValueError(f"encountered invalid format string: {text}"))
        return None

    text = re.sub(
        f"[^{re.escape(string.printable)}]",
        "",
        re.sub(
            r"[\?\.\!]+(?=[\?\.\!])",
            "",
            clean(
                text,
                fix_unicode=False,
                to_ascii=False,
                lower=False,
                no_line_breaks=True,
                no_urls=True,
                no_emails=True,
                no_punct=False,
                no_emoji=True,
                no_currency_symbols=True,
                lang="en",
                replace_with_url="",
                replace_with_email="",
            ),
        ),
    ).strip()

    for _ in punc:
        text = text.replace(_, "")
    text = text.replace(f". .", ". ").replace("  ", " ").replace('"','')
    text = text.strip()

    if len(text) < min_len:
        return "<EMPTY_TEXT>"
    else:
        return text.strip()


def load_json(fpath: str):
    if not fpath.endswith(".json"):
        raise ValueError(f"{fpath} not a json file")

    with open(fpath, "r") as fp:
        return json.load(fp)


def save_json(data, fpath: str):
    if not fpath.endswith(".json"):
        raise ValueError(f"{fpath} not a json file")

    with open(fpath, "w") as fp:
        return json.dump(data, fp)


def split_dataset(
    df: pd.DataFrame,
    test_size: float,
    id_col: str,
    stratify_col: str,
    random_state: int,
):
    x_train, x_test, y_train, y_test = train_test_split(
        df[id_col].values.tolist(),
        df[stratify_col].values.tolist(),
        stratify=df[stratify_col].values.tolist(),
        test_size=test_size,
        random_state=random_state,
    )

    df_train = df[df[id_col].isin(x_train)]
    df_test = df[df[id_col].isin(x_test)]
    df_train = df_train.reset_index()
    df_test = df_test.reset_index()

    logger.debug(
        f"\nTrain set: {df_train.shape}\n{df_train[stratify_col].value_counts()}"
    )
    logger.debug(f"\nTest Set: {df_test.shape}\n{df_test[stratify_col].value_counts()}")

    return df_train, df_test


import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


def search_best_match(search_text: str, df: pd.DataFrame, column: str):
    # Load the pre-trained sentence transformer model
    model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

    # Compute the embeddings for the search text and the DataFrame column
    search_text_embedding = model.encode([search_text])
    column_embeddings = model.encode(df[column].tolist())

    # Compute the cosine similarity between the embeddings
    similarities = cosine_similarity(search_text_embedding, column_embeddings)

    # Find the index of the best match
    best_match_index = similarities.argmax()

    # Return the best match
    return best_match_index, df.iloc[best_match_index]


import glob
import pandas as pd
import json
import ast
import re
from utils import dashed_line
import string

def clean_str(input_string)->str:
    input_string = input_string.strip() # Remove extra line spaces
    input_string = input_string.replace('\n',' </s> ') # Replace line spaces with /s<>
    input_string = input_string.replace('  ',' ')
    input_string = input_string.replace("\\",' ')
    input_string = input_string.replace("/",' ')
    input_string = input_string.lower() # Lower all characters.
    return input_string

def postprocess_llm_pred(llm_prediction:str, key="") -> str:
    """
    Function to parse the right answer from noisy LLM prediction
    """
    answer = '<None>'
    if not isinstance(llm_prediction, str) or len(llm_prediction)<1 or llm_prediction == "{}": # Return default value in case prediction == None or empty.
        return answer
    else:
        if "Question:" in llm_prediction: # For models like llama, output is both the prompt and prediction.
            llm_prediction = llm_prediction.split("Answer:")[-1]
        llm_prediction = clean_str(llm_prediction)

        if '{' in llm_prediction and '}' in llm_prediction: # Check if any json / dict object is present within the prediction.
            dict_str = extract_json(llm_prediction) # Extract dict object.
            if dict_str is not None: # Succefully extracted dict from string.
                dict_obj = str_to_dict(dict_str)
                if isinstance(dict_obj, dict) and len(dict_obj) > 0: # Succesfully converted string to dict
                    try:
                        key = list(dict_obj.keys())[0]
                        key_ = remove_punctuation(key)
                        key_ = key_.strip()
                        key_ = key_.lower()
                        dict_obj[key_] = dict_obj[key]
                    except:
                        print(dict_obj)
                    if not 'answer' in dict_obj:
                        print(
                           f"Encountered a prediction with dict object withou 'answer' key:\n{llm_prediction}"
                        )
                        return remove_punctuation(llm_prediction)

                    answer = dict_obj['answer']
                    answer = remove_punctuation(answer)
                    return answer
        
        else: # No json / dict object in prediction.
            answer = remove_punctuation(llm_prediction)
            return answer
        
    return answer

def extract_json(input_string):
    start_index = input_string.find('{')
    end_index = input_string.find('}')
    
    if start_index == -1 or end_index == -1 or end_index <= start_index:
        return None
    else:
        dict_str = "{" + input_string[start_index+1:end_index] + "}"
        return dict_str

def str_to_dict(input_string):
    input_string = input_string.strip()
    if len(input_string.split(':')) == 2: # dict object in string and not a set object
        input_string = input_string.split(':')
        key = input_string[0]
        key = key.lower().strip().replace('"','').replace("'",'').replace("{","")
        key = f'"{key}"'

        value = input_string[1]
        value = value.lower().strip().replace('"','').replace("'",'').replace("}","")
        value = f'"{value}"'

        input_string = "{" + key + ":" + value + "}"

    try:
        dict_obj = json.loads(input_string)
        if isinstance(dict_obj, set):
            dict_obj = {'answer': list(dict_obj)[0]}
        return dict_obj
    except:
        try:
            dict_obj = ast.literal_eval(input_string)
            if isinstance(dict_obj, set):
                dict_obj = {'answer': list(dict_obj)[0]}
            return dict_obj
        except:
            print(f"Could not convert the following string to dict:\n{input_string}")
            print(dashed_line)
            return None
                
def remove_punctuation(input_string, replace_by=""):
    # return re.sub(r'[^\w\s]', ' ', input_string)
    punctuations = string.punctuation
    punctuations.replace('<','')
    punctuations.replace('>','')
    punctuations.replace("'",'')
    return f'{replace_by}'.join(char for char in input_string if char not in punctuations)


def get_val(d):
    """
    Returns the value from a dictionary containing a single key-value pair.
    
    Args:
    d (dict): A dictionary with a single key-value pair.
    
    Returns:
    The value of the single key-value pair in the dictionary.
    """
    assert isinstance(d, str)
    d = ast.literal_eval(d)
    return float(next(iter(d.values())))


def extract_score(input_string: str, key='Relevance') -> float:
    if not isinstance(input_string, str):
        print(
            f"Invalid input: {input_string}"
        )
        return -1
    """
    try:
        input = ast.literal_eval(input)
        score = input[key]
        score = score.strip().replace(' ','')
        score = float(score)
        return score
    except:
        return -1
    """
     # Use regex to find the number in the string
    match = re.search(r'\d+', input_string)
    
    # If a number is found, convert it to float and return it
    if match:
        return float(match.group())
    else:
        return -1

def extract_content_from_response(response):
    try:
        return response.choices[0].message.content
    except:
        print(response)


def postprocess_llm_response(response):
    if isinstance(response, dict):
        return list(response.keys())[0]
    else:
        try:
            x = json.loads(response)
            return list(x.keys())[0]
        except:
            return str(response)

In [ ]:
# prompt templates

common_input_prompt = lambda hs, cs: f"""
Hate Speech: "{hs}"\nCounterspeech: "{cs}"
""".strip()

instruction_prompt = f"""
You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). Your
task is to rate the counterspeech on one metric. Please make sure you read and understand these instructions carefully.
""".strip()

evaluation_criteria_relevance = f"""
Contextual Relevance (1-5) - This dimension evaluates whether the counterspeech addresses the central topic, theme subject of the given hate
speech.
""".strip()

evaluation_criteria_aggressiveness = f"""
Aggressiveness (1-5) - This metric assesses the degree of confrontational or inflammatory content in the counterspeech. It considers factors such
as abusive language, intensity of disagreement, tone, engagement style, and the presence of personal attacks, with higher scores indicating
greater aggression.
""".strip()

evaluation_criteria_coherence = f"""
Argument Coherence (1-5) - This metric assesses how logically and smoothly the ideas or arguments within the counterspeech connect and
flow. A coherent counterspeech will present its arguments in an organized manner, making it easy for the reader to follow and understand the
counter-narrative being presented.
""".strip()

evaluation_criteria_suitableness = f"""
Suitableness (1-3) - This metric measures the likelihood of an annotator choosing a given counterspeech for direct use (without editing) in a
real scenario against online hate speech. This assessment considers the counterspeech's suitability, appropriateness, and potential impact on a
reader in a real-world context.
""".strip()

# ----------------------------------------------------------------------- #

relevance_score_prompt_zs = f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_relevance}

Ensure that the response is STRICTLY in JSON format as {{"Relevance": ""}}
""".strip()

coherence_score_prompt_zs = f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_coherence}

Ensure that the response is STRICTLY in JSON format as {{"Argument Coherence": ""}}.
""".strip()

suitableness_score_prompt_zs = f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_suitableness}

Ensure that the response is STRICTLY in JSON format as {{"Suitableness": ""}}.
""".strip()

aggressiveness_score_prompt_zs = f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_aggressiveness}

Ensure that the response is STRICTLY in JSON format as {{"Aggressiveness": ""}}.
""".strip()

# ----------------------------------------------------------------------- #

relevance_score_prompt_cot = lambda evaluation_steps : f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_relevance}

Evaluation Steps:
{evaluation_steps}

Ensure that the response is STRICTLY in JSON format as {{"Relevance": ""}}
""".strip()

coherence_score_prompt_cot = lambda evaluation_steps : f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_coherence}

Evaluation Steps:
{evaluation_steps}

Ensure that the response is STRICTLY in JSON format as {{"Argument Coherence": ""}}.
""".strip()

suitableness_score_prompt_cot = lambda evaluation_steps : f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_suitableness}

Evaluation Steps:
{evaluation_steps}

Ensure that the response is STRICTLY in JSON format as {{"Suitableness": ""}}.
""".strip()

aggressiveness_score_prompt_cot = lambda evaluation_steps : f"""
{instruction_prompt}

Evaluation Criteria:
{evaluation_criteria_aggressiveness}

Evaluation Steps:
{evaluation_steps}

Ensure that the response is STRICTLY in JSON format as {{"Aggressiveness": ""}}.
""".strip()

# ----------------------------------------------------------------------- #

cot_gen_few_shot_tempate = lambda aspect, hs1, cs1, r1, hs2, cs2, r2, hs3, cs3, r3: f"""
Hatespeech: {hs1}
Counterspeech: {cs1}
Expert Rating ({aspect}): {r1}

Hatespeech: {hs2}
Counterspeech: {cs2}
Expert Rating ({aspect}): {r2}

Hatespeech: {hs3}
Counterspeech: {cs3}
Expert Rating ({aspect}): {r3}
""".strip()

candidate_cot_drafting_template = lambda aspect, evaluation_criteria, few_shot_examples : f"""
You will be given sets of in-context examples, each containing a hate speech, a corresponding
counterspeech, and an {aspect} rated by a human expert. Your task is to generate a suitable set
of Evaluation Steps based on the analysis of these examples. Please make sure you read and
understand these instructions carefully.

## Examples:
{few_shot_examples}

## Instruction:
Analyze the examples below to identify patterns and factors that influence the {aspect} rating.
Then, create a detailed set of steps outlining how to evaluate the {aspect} of counterspeeches.

## Evaluation Criteria for {aspect}:
{evaluation_criteria}

""".strip()

# aspect = "Aggressiveness"
# few_shot_examples = cot_gen_few_shot_tempate(aspect,"hs1", "cs1", "r1", "hs2", "cs2", "r2", "hs3", "cs3", "r3")
# prompt = candidate_cot_drafting_template(aspect, evaluation_criteria_aggressiveness, few_shot_examples)
# print(prompt)

# ----------------------------------------------------------------------- #

cot_error_few_shot_tempate = lambda aspect, hs1, cs1, r1, p1, hs2, cs2, r2, p2, hs3, cs3, r3, p3: f"""
Hatespeech: {hs1}
Counterspeech: {cs1}
Actual Rating ({aspect}): {r1}
Predicted Rating ({aspect}): {p1}

Hatespeech: {hs2}
Counterspeech: {cs2}
Actual Rating ({aspect}): {r2}
Predicted Rating ({aspect}): {p2}

Hatespeech: {hs3}
Counterspeech: {cs3}
Actual Rating ({aspect}): {r3}
Predicted Rating ({aspect}): {p3}

""".strip()

candidate_cot_refinement_template = lambda aspect, candidate_cot, few_shot_examples: f"""
Please refine and improve the chain-of-thought (CoT) evaluation steps used by a large language model in evaluating {aspect} of counterspeech
generation.

Large language models (LLMs) are powerful neural models that can evaluate the quality of counterspeech generation. However, LLMs may not
always agree with human judgments. Please refine the CoT used by LLMs to improve its correlation with human expert scores. To refine the
scoring criteria used by the LLM in evaluating the {aspect}, please follow the following instructions step-by-step:
1. Carefully read each example, understand each hate speech and its corresponding counterspeech, and get your initial assessment of its
quality on {aspect}.
2. Compare the test score obtained by the LLM according to the CoT and the ground-truth score from human experts. Please think why the
correlation is limited by using the current CoT, and how can you improve the CoT to increase the correlation between LLM’s score and
human expert score. If there is a small gap or no gap, this means the CoT work well in this case.
3. Read all of the test cases and rethink how you could refine the current CoT based on your observations and analysis. Then, refine the
CoT to make it concise, accurate, and consistent with human judgments. When refining the CoT, you can do the following: 1)
modification: adjust some parts of the CoT to increase its correlation with the scoring CoT that you think might used by human experts;
2) paraphrase: if the CoT is good enough, you can consider paraphrasing it to make more concise and easy to understand; 3) adding
aspects or details: if you find some new underlying scoring rules not covered by the current CoT, consider adding them as a new line of
injecting to current CoT, but make sure not to make the CoT too long and redundant; 4) calibrate: you can take other methods you think
being helpful to improve the correlation with human experts.
Please return only your refined criteria without any additional sentences.

Old criteria:
{candidate_cot}

Error Examples:
{few_shot_examples}
""".strip()

# aspect = "Aggressiveness"
# candidate_cot = "Rate 1 as most aggressive and 5 as least aggressive"
# few_shot_examples = cot_error_few_shot_tempate(aspect, "hs1", "cs1", "r1", "p1", "hs2", "cs2", "r2", "p2", "hs3", "cs3", "r3", "p3")
# prompt = candidate_cot_refinement_template(aspect, candidate_cot, few_shot_examples)
# print(prompt)


In [ ]:
df_dev = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dev_set_500.csv')

dev_set_gold_relevance_score = []
dev_set_gold_coherence_score = []
dev_set_gold_aggressiveness_score = []
dev_set_gold_suitableness_score = []

for i, row in df_dev.iterrows():
    dev_set_gold_relevance_score.append(
        (row['hatespeech'], row['predicted_counterspeech'], row['relevance_score'])
    )
    dev_set_gold_coherence_score.append(
        (row['hatespeech'], row['predicted_counterspeech'], row['coherence_score'])
    )
    dev_set_gold_aggressiveness_score.append(
        (row['hatespeech'], row['predicted_counterspeech'], row['aggressiveness_score'])
    )
    dev_set_gold_suitableness_score.append(
        (row['hatespeech'], row['predicted_counterspeech'], row['suitableness_score'])
    )


In [ ]:
df_dev.head(3)

In [ ]:
# def generate_candidate_cot(criteria, system_description=""):
#     prompt = criteria
#     return f"candidate cot {len(prompt)}"

# def generate_candidate_cot_self_refinement(refinement_prompt, system_description=""):
#     prompt = refinement_prompt
#     return f"refined candidate cot {len(prompt)}"



In [ ]:
import random
from typing import List, Tuple

# Toy values for the golden set D*
# dev_set_gold = [
#     ("Article A1", "Summary S1", 5),
#     ("Article A2", "Summary S2", 4),
#     ("Article A3", "Summary S3", 3),
#     ("Article A4", "Summary S4", 2),
#     ("Article A5", "Summary S5", 1)
# ]

dev_set_gold = dev_set_gold_relevance_score
aspect = "Relevance"
evaluation_criteria = evaluation_criteria_relevance

# Number of Monte Carlo trials
num_trials = 10

# Few-shot exemplar size
few_shot_size = 3

# Define topk
topk = 3


# Perform Monte Carlo trials
criteria_set = []
candidate_cot_set = []

for trial in range(num_trials):
    print(f"Trial {trial + 1}:")

    # Randomly sample few-shot examples from the golden set
    sampled_examples = random.sample(dev_set_gold, few_shot_size)

    # Infer criteria based on sampled examples
    hs1, cs1, r1 = sampled_examples[0][0], sampled_examples[0][1], sampled_examples[0][2]
    hs2, cs2, r2 = sampled_examples[1][0], sampled_examples[1][1], sampled_examples[1][2]
    hs3, cs3, r3 = sampled_examples[2][0], sampled_examples[2][1], sampled_examples[2][2]

    few_shot_examples = cot_gen_few_shot_tempate(aspect, hs1, cs1, r1, hs2, cs2, r2, hs3, cs3, r3)
    criteria = candidate_cot_drafting_template(aspect, evaluation_criteria, few_shot_examples)
    
    # Store the inferred criteria
    criteria_set.append(criteria)

    candidate_cot = predict(prompt=criteria, system_description="", use_temperature_samping=True)
    print(candidate_cot)
    candidate_cot = postprocess_llm_response(candidate_cot)
    candidate_cot_set.append(candidate_cot)
    
    print(f"Sampled Examples: {sampled_examples}")
    print(f"Inferred Criteria: {criteria}")
    print(f"Candidate COT: {candidate_cot}")


In [ ]:
from metric import spearman_correlation, kendall_tau
import json


def postprocess_llm_response(response):
    if isinstance(response, dict):
        return list(response.keys())[0]
    else:
        try:
            x = json.loads(response)
            return list(x.keys())[0]
        except:
            return str(response)


def evaluate_candidate_cot(aspect, candidate_cot, test_size=10) -> float:
    # Returns average correlation scores for each candidate cot.
    
    print(f"Total candidates to evaluate: {len(candidate_cot_set)}")
    scoring_prompt = None
    actual_scores = []
    predicted_scores = []
    exemplars = []

    if aspect == "Relevance":
        scoring_prompt = relevance_score_prompt_cot(evaluation_steps=candidate_cot)
    elif aspect == "Aggressiveness":
        scoring_prompt = aggressiveness_score_prompt_cot(evaluation_steps=candidate_cot)
    elif aspect == "Coherence":
        scoring_prompt = coherence_score_prompt_cot(evaluation_steps=candidate_cot)
    elif aspect == "Suitableness":
        scoring_prompt = suitableness_score_prompt_cot(evaluation_steps=candidate_cot)

    print(f"Evaluating on {test_size} datapoints")
    for i, row in tqdm(df_dev[:test_size].iterrows(), desc="Getting Predictions"):
        input_prompt = common_input_prompt(row['hatespeech'], row['predicted_counterspeech'])
        system_description = scoring_prompt

        response = predict(
            prompt=input_prompt, 
            system_description=system_description
        )
        predicted_score = extract_score(response)
        predicted_scores.append(predicted_score)

        if aspect == "Relevance":
            actual_score = row["relevance_score"]
        elif aspect == "Aggressiveness":
            actual_score = row["aggressiveness_score"]
        elif aspect == "Coherence":
            actual_score = row["coherence_score"]
        elif aspect == "Suitableness":
            actual_score = row["suitableness_score"]
        
        actual_scores.append(actual_score)

        exemplars.append(
            (row['hatespeech'], row['predicted_counterspeech'], actual_score, predicted_score)
        )

    spearman_score = spearman_correlation(actual_scores, predicted_scores)
    kendalltau_score = kendall_tau(actual_scores, predicted_scores)
    mean_score = (spearman_score + kendalltau_score)/2
    return (mean_score, exemplars)

evaluated_candidates = [(candidate_cot, evaluate_candidate_cot(aspect, candidate_cot)[0], evaluate_candidate_cot(aspect, candidate_cot)[1]) for candidate_cot in candidate_cot_set]
evaluated_candidates.sort(key=lambda x: x[1], reverse=True)
top_candidates = evaluated_candidates[:topk]

print("Top-performing Criteria:")
for candidate_cot, score, exemplars in top_candidates:
    print(f"Candidate COT: {candidate_cot}, Score: {score}, Exempars: {exemplars}")

In [ ]:
def iterative_refinement_of_cot_candidate(candidate_cot, aspect) -> list[str]:
    # Performs interative refinement of top candidate via self-assessement prompting, and returns topk candidates with highest correlation
    exemplars = candidate_cot[2]
    refinement_prompts = []
    candidate_cot_set = []

    # Number of Monte Carlo trials
    num_trials = 3

    # Few-shot exemplar size
    few_shot_size = 3


    for trial in range(num_trials):
        print(f"Trial {trial + 1}:")

        # Randomly sample few-shot examples from the golden set
        sampled_examples = random.sample(exemplars, few_shot_size)

        # Infer criteria based on sampled examples
        hs1, cs1, r1, p1 = sampled_examples[0][0], sampled_examples[0][1], sampled_examples[0][2], sampled_examples[0][3]
        hs2, cs2, r2, p2 = sampled_examples[1][0], sampled_examples[1][1], sampled_examples[1][2], sampled_examples[1][3]
        hs3, cs3, r3, p3 = sampled_examples[2][0], sampled_examples[2][1], sampled_examples[2][2], sampled_examples[2][3]

        few_shot_examples = cot_error_few_shot_tempate(aspect, hs1, cs1, r1, p1, hs2, cs2, r2, p2, hs3, cs3, r3, p3)
        cot_refinement_prompt = candidate_cot_refinement_template(aspect, candidate_cot[0], few_shot_examples)
        # print(cot_refinement_prompt)

        refinement_prompts.append(cot_refinement_prompt)

        candidate_cot_refined = predict(prompt=cot_refinement_prompt, system_description="")
        candidate_cot_refined = postprocess_llm_response(candidate_cot_refined)
        candidate_cot_set.append(candidate_cot_refined)
        
    evaluated_candidates = [(candidate_cot, evaluate_candidate_cot(aspect, candidate_cot)[0], evaluate_candidate_cot(aspect, candidate_cot)[1]) for candidate_cot in candidate_cot_set]
    evaluated_candidates.sort(key=lambda x: x[1], reverse=True)
    top_candidate = evaluated_candidates[0]
    return top_candidate


top_candidates_refined = []
for candidate_cot in top_candidates:
    print(candidate_cot)
    refined_cot_candidate = iterative_refinement_of_cot_candidate(candidate_cot, aspect)
    top_candidates_refined.append(refined_cot_candidate)

top_candidates_refined.sort(key=lambda x: x[1], reverse=True)
top_candidates = top_candidates_refined[:topk]
top_candidate_global = top_candidates_refined[0]
top_candidate_global